In [4]:
import numpy as np
import matplotlib.pyplot as pl
import qnm_filter
import qnm
import matplotlib
import astropy.constants as c
T_MSUN = c.M_sun.value * c.G.value / c.c.value**3
mplparams = {
    'text.usetex': True,
    'axes.linewidth': 1,
    'lines.linewidth' : 1.5,
    'axes.grid': False,
    'axes.labelweight': 'normal',
    'font.family': 'DejaVu Sans',
    'font.size': 26,
    'figure.figsize': (10,8),
    'legend.fontsize': 26,
    'legend.handlelength' : 2,
    'legend.numpoints' : 1,
    'axes.grid' : True,
    'grid.alpha' : 0.3,
}
matplotlib.rcParams.update(mplparams)

red = '#D60606'
blue = '#0083DE'
green = '#00BA75'
yellow = '#FFC61E'
purple = '#A433B3'
orange = '#FD882E'
color_arr = [red, blue, green, yellow, purple, orange]

In [6]:
srate = 4096*4
duration = 2
t_range=np.arange(-duration/2,duration/2,1/srate)
IMR = [60, 0.7]
mass = qnm_filter.Filter.mass_unit(IMR[0])
omega220 = qnm.modes_cache(s=-2, l=2, m=2, n=0)(a=IMR[1])[0]
omega221 = qnm.modes_cache(s=-2, l=2, m=2, n=1)(a=IMR[1])[0]

mmax = 8.4 * 1e-21
phase1 = 0 #random.uniform(0, 2 * np.pi)
A220x = mmax * np.cos(phase1)
A220y = mmax * np.sin(phase1)
phase2 = 0 #random.uniform(0, 2 * np.pi)
A221x = mmax * np.cos(phase2)
A221y = mmax * np.sin(phase2)

power220 = 1 #random.uniform(0.005, 0.3)
power221 = 1 #random.uniform(0.005, 0.3)
amp220 = np.sqrt(power220)
amp221 = np.sqrt(power221)
signal220 = np.real(amp220 * (A220x + 1j*A220y) * np.exp(-1j * omega220 * np.abs(t_range / mass)))
signal221 = np.real(amp221 * (A221x + 1j * A221y) * np.exp(-1j * omega221 * np.abs(t_range / mass)))
signal = signal220 #+ signal221

# #Loop use to be here:
# bilby_ifo = qnm_filter.set_bilby_predefined_ifo(
# "H1", sampling_frequency, duration, start_time=-duration / 2
# )
# signalH_noise = qnm_filter.bilby_get_strain(bilby_ifo, 0.0)	
# signalH_no_noise = qnm_filter.RealData(signal, index=t_range, ifo="H1")
# signalH = signalH_no_noise + signalH_noise

In [ ]:
input = dict(model_list = [(2,2,0, 'p')],
             t_init = 0,
             segment_length = 0.2,
             srate = srate,
             ra = None, dec = None,)

fit = qnm_filter.Network(segment_length=0.2, srate=4096 * 1, t_init=3.0 * mass)
fit.original_data["H1"] = signalH
fit.detector_alignment()
fit.pure_noise = {}
fit.pure_noise["H1"] = signalH_noise
fit.pure_nr = {}
fit.pure_nr["H1"] = signalH_no_noise

fit.condition_data("original_data")
fit.condition_data("pure_noise")
fit.condition_data("pure_nr")
fit.compute_acfs("pure_noise")
fit.cholesky_decomposition()
fit.first_index()

data = fit.truncate_data(fit.original_data)['H1']
fit.pure_nr = {}

fit.pure_nr["H1"] = qnm_filter.RealData(signal220+signal221, index=t_range, ifo="H1")
fit.condition_data("pure_nr")
template_both = fit.truncate_data(fit.pure_nr)["H1"]

fit.pure_nr["H1"] = qnm_filter.RealData(signal220, index=t_range, ifo="H1")
fit.condition_data("pure_nr")
template_220 = fit.truncate_data(fit.pure_nr)["H1"]

fit.pure_nr["H1"] = qnm_filter.RealData(signal221, index=t_range, ifo="H1")
fit.condition_data("pure_nr")
template_221 = fit.truncate_data(fit.pure_nr)["H1"]